# Прогнозирование временных рядов

Прогнозирование временных рядов — это важная задача в аналитике данных, которая помогает предсказывать будущее поведение данных на основе исторических данных. В этом блоке мы рассмотрим основы анализа временных рядов, изучим, как работать с ними в Python, и познакомимся с основными методами оценки точности прогнозов.

---

## Введение в анализ временных рядов

### Что такое временные ряды и их использование

**Временные ряды** — это последовательность наблюдений, записанных через равные промежутки времени. Они широко используются для анализа данных, которые зависят от времени, таких как прогнозирование продаж, температуры, финансовых показателей, спроса на продукцию и т.д.

Примеры применения временных рядов:
- **Финансовый анализ**: прогнозирование цен на акции или обменных курсов.
- **Производственные процессы**: прогнозирование объёмов выпуска продукции.
- **Экономика**: анализ и прогнозирование макроэкономических показателей.
- **Прогнозирование спроса**: планирование запасов на основе данных о продажах.

### Основные методы прогнозирования

1. **Простое сглаживание (Simple Moving Average)**:
   - Метод заключается в усреднении ряда данных за определённый промежуток времени.
   - Пример: среднее значение продаж за последние 3 месяца.

2. **Экспоненциальное сглаживание (Exponential Smoothing)**:
   - Усреднение данных с учётом весов, придаваемых более свежим данным.
   - Пример: модель **Holt-Winters**, которая учитывает как тренд, так и сезонность данных.

3. **ARIMA (AutoRegressive Integrated Moving Average)**:
   - Популярная модель для прогнозирования временных рядов, которая включает авторегрессию (AR), интегрированную компоненту (I) и скользящее среднее (MA).
   - Применяется для прогнозирования данных с трендами и сезонными колебаниями.

4. **Prophet**:
   - Модель, разработанная Facebook, специально для работы с временными рядами. Хорошо справляется с данными, содержащими тренды, сезонность и аномалии.

5. **TSFresh и Pmdarima**:
   - **TSFresh** используется для извлечения признаков из временных рядов для дальнейшего анализа и машинного обучения.
   - **Pmdarima** автоматически подбирает параметры ARIMA и выполняет прогнозирование.

---

## Работа с временными рядами в Python

#### Введение в библиотеки Pandas для работы с временными рядами

**Pandas** предоставляет множество удобных инструментов для работы с временными рядами. Среди них:
- Индексация по дате и времени.
- Вычисление скользящего среднего.
- Ресемплирование данных (например, агрегирование по дням, неделям, месяцам).

##### Пример работы с временными рядами в Pandas:

```python
import pandas as pd
import numpy as np

# Загрузка данных
df = pd.read_excel('timeseries.xlsx', index_col='date', parse_dates=['date'])

# Вывод первых строк с индексом времени
df.head()
```

Вывод графика

```python
# Построение графика временного ряда
df['value'].plot(title='Объем, тыс. тонн', figsize=(17,4))
```

### Пояснение:
- **`parse_dates=['Date']`** — преобразует столбец с датами в формат `datetime`.
- **`index_col='Date'`** — делает столбец с датами индексом DataFrame.
- **`plot()`** — построение графика временного ряда.

## Поиск выбросов во временных рядах с учётом сезонных колебаний

Временные ряды могут иметь как случайные выбросы, так и выбросы, связанные с сезонными колебаниями. Выбросы, не принимающие во внимание сезонность, могут ошибочно считаться аномалиями. В этом блоке рассмотрим, как искать выбросы во временных рядах с учётом сезонных колебаний и как корректировать их.

### Шаг 1: Поиск выбросов с учётом сезонности

Для корректного поиска выбросов во временных рядах с выраженной сезонностью часто применяются специализированные методы.

Декомпозиция позволяет разделить временной ряд на три компонента:
- **Тренд** — долгосрочные изменения во времени.
- **Сезонность** — повторяющиеся циклические колебания.
- **Остатки** — случайные флуктуации и выбросы.

Сначала разделим временной ряд с помощью декомпозиции, а затем поищем выбросы в остатках.

```python
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import STL

# Функция для добавления графиков STL
def add_stl_plot(fig, res, legend):
    axs = fig.get_axes()
    comps = ["trend", "seasonal", "resid"]
    for ax, comp in zip(axs[1:], comps):
        series = getattr(res, comp)
        if comp == "resid":
            ax.plot(series, marker="o", linestyle="none")
        else:
            ax.plot(series)
            if comp == "trend":
                ax.legend(legend, frameon=False)

# Декомпозиция временного ряда с использованием STL
stl = STL(df['value'], period=12, robust=True)
res_robust = stl.fit()
fig = res_robust.plot()
res_non_robust = STL(df['value'], period=12, robust=False).fit()
add_stl_plot(fig, res_non_robust, ["С выбросами", "Очищенный от выбросов"])
```

На итоговом графике мы увидим результаты как изначального ряда данных, так и очищенного. 

### Пояснение:
- **`seasonal_decompose()`** — функция для декомпозиции временного ряда на тренд, сезонность и остатки. 
- **Остатки** содержат случайные флуктуации и выбросы, поэтому именно они используются для поиска аномалий.

#### Обнаружим выбросы

Код ниже выведет периоды, которые содержат выбросы. А также остатки, те разницу между фактическим значением и рассчитанной величиной.

```python
# Обнаружение выбросов на основе остатков (residuals)
residuals = res_robust.resid
Q1 = residuals.quantile(0.25)
Q3 = residuals.quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
outliers = residuals[(residuals < lower_bound) | (residuals > upper_bound)]

# Вывод выбросов
print('Выбросы, остатки:')
print(outliers)
```

#### Коррекция данных

Код ниже создаст в исходном DataFrame новый столбец `value_corrected`, куда поместит скорректированные значения для периодов с выбросами. Новые значение будут рассчитаны как тренд плюс сезонная компонента.

```python
# Замена выбросов на значения, равные сумме тренда и сезонной компоненты
df['trend'] = res_robust.trend
df['seasonal'] = res_robust.seasonal
df['value_corrected'] = df['value'].copy()

# Замена выбросов
for idx in outliers.index:
    df.loc[idx, 'value_corrected'] = df.loc[idx, 'trend'] + df.loc[idx, 'seasonal']

# Визуализация исходного и скорректированного временного ряда
plt.figure(figsize=(14, 7))
plt.plot(df.index, df['value'], label='Исходные данные', linestyle='-', marker='o')
plt.plot(df.index, df['value_corrected'], label='Скорректированные данные', linestyle='--', marker='x')
plt.xlabel('Дата')
plt.ylabel('Значение')
plt.title('Сравнение исходных и скорректированных данных')
plt.legend()
plt.grid(True)
plt.show()
```

#### Шаг 2. Использование логарифмического преобразования

Логарифмическое преобразование может уменьшить влияние выбросов, особенно когда выбросы значительно отклоняются по величине. Может применяться в рядах как с сезонными компонентами и трендом, так и не содержащим периодических закономерностей. 

```python
# Применение логарифмического преобразования к данным
df['Log_value'] = np.log(df['value'])
df['Log_value'].plot()
```

#### Рекомендации:
- Логарифмическое преобразование часто помогает, если выбросы пропорциональны значению переменной и связаны с изменением масштаба.

Чтобы вернуть результаты логорифмического преобразования к изначальной размерности, можно выполнить следующий код:

```python
# вернуть к исходной размерности
norm_data=df['Log_value']
отобразим на графике
np.exp(df['Log_value']).plot()
```

### Заключение

Для поиска выбросов в сезонных временных рядах лучше всего использовать методы, которые учитывают структуру данных, такие как декомпозиция или скользящее среднее с сезонным окном. Исправление выбросов может включать удаление аномалий, замену сезонными средними значениями или использование логарифмического преобразования для сглаживания отклонений. 

## Использование библиотеки Prophet для прогнозирования

Далее мы работаем со скорректированными данными.

Prophet — это мощная библиотека для прогнозирования временных рядов, которая автоматически учитывает тренды и сезонность данных.

##### Установка Prophet:

```bash
!pip -q install prophet
```

##### Пример использования Prophet:

```python
from prophet import Prophet

# Подготовка данных для Prophet
df_prophet = df.reset_index().rename(columns={'date': 'ds', 'value_corrected': 'y'})
df_prophet['ds'] = pd.to_datetime(df_prophet['ds'])

# Создание модели Prophet
model = Prophet()
model.fit(df_prophet)

# Прогнозирование на следующие периоды
future = model.make_future_dataframe(periods=12, freq='M')
forecast = model.predict(future)

# Визуализация прогноза
model.plot(forecast)
plt.show()
```

### Пояснение:
- **`ds`** — стандартное имя для столбца с датами в Prophet.
- **`y`** — значение временного ряда, которое нужно прогнозировать.
- **`make_future_dataframe(periods=30)`** — создаёт прогноз на следующие 30 дней.

Посмотреть прогноз можно вызвав переменную `forecast`

Или сохранить в Excel файл

```python
forecast.to_excel('forecast_2025.xlsx')
```

## Модель Хольта-Винтерса

Модель Хольта-Винтерса (Holt-Winters) — это метод временного ряда, который используется для прогнозирования данных с трендом и сезонностью. Давайте рассмотрим пример, как использовать эту модель для прогнозирования.

```python
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.holtwinters import ExponentialSmoothing

# Обучение модели Хольта-Винтерса
df=df.resample('ME').mean()
model = ExponentialSmoothing(df['value_corrected'], seasonal='mul', seasonal_periods=12, use_boxcox=False)
hw_model = model.fit(optimized=True, remove_bias=False)

# Прогнозирование на следующие 12 месяцев
forecast_hw = hw_model.forecast(12)

# Визуализация результатов
plt.figure(figsize=(12, 6))
plt.plot(df.index, df['value_corrected'], label='Исторические данные')
plt.plot(pd.date_range(start=df.index[-1] + pd.DateOffset(1), periods=12, freq='ME'), forecast_hw, label='Прогноз', color='orange')
plt.xlabel('Дата')
plt.ylabel('Значение')
plt.title('Прогноз с использованием модели Хольта-Винтерса')
plt.legend()
plt.show()
```

**Объяснение кода:**

Импорт библиотек: Используем pandas для работы с временными рядами, numpy для генерации данных, matplotlib для визуализации и ExponentialSmoothing из statsmodels для применения модели Хольта-Винтерса.

Обучение модели Хольта-Винтерса:
- Параметр seasonal='add' указывает, что сезонность в данных аддитивная. Если ваша сезонность мультипликативная, используйте seasonal='mul'.
- seasonal_periods=12 указывает, что сезонность повторяется каждые 12 месяцев.

Прогнозирование: Используем метод forecast для предсказания на 12 будущих периодов.

Визуализация: Отображаем как исторические данные, так и прогноз, чтобы увидеть, как модель предсказывает будущие значения.

Этот пример демонстрирует, как использовать модель Хольта-Винтерса для прогнозирования временных рядов с сезонностью.

## Оценка моделей временных рядов

После построения модели важно оценить её точность, используя различные метрики. В прогнозировании временных рядов наиболее часто применяются следующие метрики:

1. **MAE (Mean Absolute Error)** — средняя абсолютная ошибка. Измеряет среднее абсолютное отклонение прогнозов от фактических значений.

   Формула:
   $$
   MAE = \frac{1}{n} \sum_{i=1}^{n} |y_i - \hat{y}_i|
   $$

2. **RMSE (Root Mean Squared Error)** — среднеквадратичная ошибка. Оценка средней ошибки прогнозирования с увеличенным влиянием больших отклонений.

   Формула:
   $$
   RMSE = \sqrt{\frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2}
   $$

3. **MAPE (Mean Absolute Percentage Error)** — средняя абсолютная процентная ошибка. Показывает, на сколько процентов в среднем прогноз отличается от фактического значения.

   Формула:
   $$
   MAPE = \frac{1}{n} \sum_{i=1}^{n} \left( \frac{|y_i - \hat{y}_i|}{y_i} \right) \times 100
   $$

#### Пример расчёта метрик:

```python
from sklearn.metrics import mean_absolute_error, root_mean_squared_error

# Истинные значения и прогнозы (например, для Prophet)
y_true = df['value_corrected'][-12:].values  # последние 30 значений для оценки
y_pred = forecast['yhat'][-24:-12].values  # прогнозируемые значения Prophet

# MAE
mae = mean_absolute_error(y_true, y_pred)

# RMSE
rmse = root_mean_squared_error(y_true, y_pred)

# MAPE
mape = (abs((y_true - y_pred) / y_true)).mean() * 100

# Вывод метрик
print(f'MAE: {mae}')
print(f'RMSE: {rmse}')
print(f'MAPE: {mape}%')
```

### Пояснение:
- **`mean_absolute_error()`** — вычисляет среднюю абсолютную ошибку.
- **`mean_squared_error(squared=False)`** — вычисляет RMSE.
- **`MAPE`** — рассчитывается вручную, поскольку она не встроена в `sklearn`.

### Для Хольта-Винтерса

Чтобы рассчитать метрики ошибки, такие как **MAE** (Mean Absolute Error), **MAPE** (Mean Absolute Percentage Error), и **RMSE** (Root Mean Squared Error) без разделения данных на обучающие и тестовые, можно использовать следующую стратегию:

1. **Сравнить прогнозируемые значения с последними известными данными в исходной серии**. Для этого вы можете взять последние 12 значений из временного ряда и использовать их в качестве "тестовых" данных для оценки точности модели.

### Пример кода для расчета метрик ошибки:

```python
from sklearn.metrics import mean_absolute_error, mean_squared_error, root_mean_squared_error

# Предположим, что у вас есть 12 последних известных значений для сравнения
y_true = df['value_corrected'][-12:]  # Последние 12 известных значений в исходном временном ряде
y_pred = hw_model.fittedvalues[-12:]  # Соответствующие предсказания модели

# Рассчитаем метрики ошибки
mae = mean_absolute_error(y_true, y_pred)
mse = root_mean_squared_error(y_true, y_pred)
mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# Вывод результатов
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Root Mean Squared Error (RMSE): {mse:.2f}")
print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%")
```

### Объяснение:
1. **y_true**: Это последние 12 известных значений из вашего временного ряда, которые мы используем для сравнения.
2. **y_pred**: Это предсказанные значения, полученные моделью Хольта-Винтерса. Мы используем `hw_model.fittedvalues` для извлечения предсказаний, которые соответствуют историческим данным.
3. **MAE, RMSE, MAPE**: Вычисляются с использованием библиотек `sklearn` и `numpy`.

### Примечание:
- `hw_model.fittedvalues` предоставляет предсказания модели на основе обучающих данных. Мы сравниваем последние известные значения с соответствующими предсказаниями.
- Этот подход позволяет оценить точность модели, даже если вы не разделяли данные явно на тренировочные и тестовые.

Теперь вы сможете получить оценку ошибок модели и понять, насколько хорошо она справляется с прогнозированием.

## Использование библиотеки Pmdarima для ARIMA

Pmdarima — это удобная библиотека для работы с моделями временных рядов, которая автоматизирует процесс построения моделей ARIMA. Она упрощает выбор оптимальных параметров модели с помощью функции auto_arima, что значительно облегчает анализ временных рядов.

**Pmdarima** автоматизирует процесс выбора параметров для модели ARIMA.

##### Установка Pmdarima:

```bash
!pip install -q pmdarima
```

##### Пример использования Pmdarima:

```python
import pandas as pd
import matplotlib.pyplot as plt
from pmdarima import auto_arima


# Обучение модели ARIMA
model_arima = auto_arima(
    df['value_corrected'],
    start_p=1, start_q=1,
    max_p=5, max_q=5,
    seasonal=True, m=12,
    d=None,  # Автоматическое определение дифференцирования
    stepwise=False,  # Перебор всех вариантов
    trace=True,  # Вывод информации о процессе перебора
    error_action='ignore',
    suppress_warnings=True
)

# Прогнозирование на следующие 12 периодов
forecast_arima = model_arima.predict(n_periods=12)

# Создание индексов для прогнозов
forecast_index = pd.date_range(start=df.index[-1] + pd.DateOffset(1), periods=12, freq='M')

# Создание DataFrame для прогнозов
forecast_df = pd.DataFrame(forecast_arima, index=forecast_index, columns=['forecast'])

# Объединение исходных данных и прогнозов в один DataFrame
combined_df = pd.concat([df, forecast_df])

# Визуализация исходных данных и прогнозов
plt.figure(figsize=(14, 7))
plt.plot(df.index, df['value_corrected'], label='Исходные данные', linestyle='-', marker='o')
plt.plot(forecast_df.index, forecast_df['forecast'], label='Прогноз', linestyle='--', marker='x', color='red')
plt.xlabel('Дата')
plt.ylabel('Значение')
plt.title('Сравнение исходных данных и прогнозов')
plt.legend()
plt.grid(True)
plt.show()
```

### Пояснение:
- **`auto_arima()`** — автоматически подбирает параметры модели ARIMA.
- **`seasonal=True`** и **`m=12`** — задают сезонность данных с периодом 12 (например, месячные данные).

In [ ]:
!pip install numpy==1.26.4

Подсчитаем метрики точности на основании сравнения результатов модели и последнего известного года.

```python
from sklearn.metrics import mean_absolute_error, mean_squared_error, root_mean_squared_error

#вернет значения модели для известных периодов
forecast_arima_ = model_arima.predict_in_sample()

# Предположим, что у вас есть 12 последних известных значений для сравнения
n_test = 12  # Количество значений для тестирования
y_true = df['value_corrected'][-n_test:]  # Последние 12 известных значений в исходном временном ряде
y_pred = forecast_arima_[-n_test:]  # Соответствующие предсказания модели

# Рассчитаем метрики ошибки
mae = mean_absolute_error(y_true, y_pred)
mse = root_mean_squared_error(y_true, y_pred)
mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# Вывод результатов
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Root Mean Squared Error (RMSE): {mse:.2f}")
print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%")
```

Взглянем, как расчетное значение описывает фактическое.

```python
# Визуализация исходного и скорректированного временного ряда
plt.figure(figsize=(14, 7))
plt.plot(df.index, df['value_corrected'], label='Исходные данные', linestyle='-', marker='o')
plt.plot(df.index, forecast_arima_, label='Скорректированные данные', linestyle='--', marker='x')
plt.xlabel('Дата')
plt.ylabel('Значение')
plt.title('Сравнение исходных и скорректированных данных')
plt.legend()
plt.grid(True)
plt.show()
```

## Заключение

1. **Основы временных рядов**: Временные ряды используются для анализа данных, зависящих от времени, с использованием различных методов прогнозирования (от простого сглаживания до сложных моделей).
2. **Работа с временными рядами в Python**: Мы рассмотрели библиотеки Pandas для работы с временными рядами и такие инструменты для прогнозирования, как Prophet и Pmdarima.
3. **Оценка точности моделей**: MAE, RMSE и MAPE — основные метрики, используемые для оценки качества прогноза.

Эти знания помогут вам эффективно анализировать и прогнозировать временные ряды.

# Задание

Выберите в датасете иной ряд для прогнозирования (другую группу грузов).

Проанализируйте, очистите ряд и постройте прогноз двумя методами. 

Сравните результаты и выберите наиболее точный. 